# CHAMPS Scalar Coupling: Medal Plan

Objectives:
- Build a reliable CV pipeline and strong feature set; target ≥ bronze.
- Ship a baseline quickly, iterate with OOF-driven improvements.

Plan:
1) Environment & GPU check (exit if no GPU).
2) Data loading + schema/EDA (train/test/structures + aux tables).
3) Validation: GroupKFold by molecule (molecule_name) with fixed seeds; save folds.
4) Baseline features:
   - Core: atom indices/types, type (coupling type), distance and geometry features from structures.
   - Per-atom environment: neighbor counts, bond lengths (kNN), angles, same-atom flags.
   - Global molecule stats; link aux tables (shielding, Mulliken, dipole, energy) by molecule/atom.
5) Models:
   - Start with CatBoost/XGBoost (GPU) per coupling type model; log progress + early stopping.
   - Tune key params (depth/eta/l2) after baseline; blend per-type models.
6) Metric & tracking:
   - Compute log-MAE on OOF per type and overall; cache OOF/test preds.
7) Iterations:
   - FE v1: simple geometry (distances, 3D deltas).
   - FE v2: neighbor-based aggregates, angles/dihedrals.
   - FE v3: merge physics tables; careful fold-only merges.
   - Ensembling of diverse seeds/algos if time.
8) Deliverables:
   - submission.csv from best CV; sanity-check file shape and IDs.

Checkpoints requiring expert review:
- After plan (now), after EDA/schema, after baseline CV, after FE v2, after tuning/ensembling.

In [1]:
# Environment & GPU check
import subprocess, sys, time
print('Checking GPU with nvidia-smi...', flush=True)
t0 = time.time()
proc = subprocess.run(['bash','-lc','nvidia-smi -L'], capture_output=True, text=True)
print(proc.stdout.strip())
print(proc.stderr.strip())
ok = (proc.returncode == 0) and ('GPU' in proc.stdout)
print(f'nvidia-smi ok: {ok} | elapsed: {time.time()-t0:.2f}s', flush=True)
if not ok:
    print('FATAL: GPU not available. Exiting to avoid wasting time.', file=sys.stderr, flush=True)
    raise SystemExit(1)

Checking GPU with nvidia-smi...


Failed to initialize NVML: Unknown Error

nvidia-smi ok: False | elapsed: 0.01s


FATAL: GPU not available. Exiting to avoid wasting time.


SystemExit: 1

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
